## Bachelor Arbeit:

Wie stark sind die Auswirkungen von Transfer Learning bezüglich der Effizienz und Effektivität anhand des Beispiels Autokennzeichen, wenn die erhebbaren Daten begrenzt sind?
***
How strong is the impact of transfer learning regarding the efficiency and effectiveness of convolutional neural networks, based on the example of detecting vehicle registration plates, assuming working with limited datasets?

In [14]:
# install required packages
!pip install fiftyone

#### Dataset:

Zuerst werden in den folgenden Codezeilen die Trainings-, Test- und Validationsdaten der Open Images Dataset Datenbank (https://storage.googleapis.com/openimages/web/visualizer/index.html?type=detection&set=train&c=%2Fm%2F01jfm_) heruntergeladen. Dazu wird die von Google empfohlene FiftyOne Bibliothek verwendet (https://voxel51.com/docs/fiftyone/).

In [13]:
# download data from Open Image Dataset V6
import os
import fiftyone

data_dir = os.path.join(os.getcwd(), 'data')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
dataset = fiftyone.zoo.load_zoo_dataset(
    "open-images-v6",
    label_types="detections",
    classes=["Vehicle registration plate"],
    max_samples=100,
    dataset_dir=data_dir
)

C:\Users\Benni\Desktop\PythonProjects\BA\bachelor-thesis-transfer-learning\code\..
C:\Users\Benni\Desktop\PythonProjects\BA\bachelor-thesis-transfer-learning\code\..\data
 100% |██████|    4.8Gb/4.8Gb [1.2m elapsed, 0s remaining, 50.6Mb/s]      
 100% |███████████████████| 100/100 [7.7s elapsed, 0s remaining, 15.9 files/s]      
 100% |███████████████████| 100/100 [7.7s elapsed, 0s remaining, 15.2 files/s]      
 100% |███████████████████| 100/100 [7.4s elapsed, 0s remaining, 15.5 files/s]      
Dataset info written to 'C:\Users\Benni\Desktop\PythonProjects\BA\bachelor-thesis-transfer-learning\code\..\data\info.json'
Loading existing dataset 'open-images-v6-100'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [ ]:
#for evaluation https://voxel51.com/docs/fiftyone/user_guide/evaluation.html#evaluating-detections-open-images ?
results = dataset.evaluate_detections("predictions", gt_field="detections", method="open-images")